In [311]:
import os
import numpy as np
import pandas as pd
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [312]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

In [313]:


DB_HOST = os.getenv('GAUSSDB_HOST')
DB_PORT = os.getenv('GAUSSDB_PORT')
DB_NAME = os.getenv('GAUSSDB_DB_SILVER')
DB_USER = os.getenv('GAUSSDB_USER')
DB_PASSWORD = os.getenv('GAUSSDB_PASSWORD')
DB_SSLMODE = os.getenv('GAUSSDB_SSLMODE')

conn_str = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

query = "SELECT * FROM energy_consumption_hourly"

df = pd.read_sql(query, conn)

conn.close()

C:\Users\Abouda\AppData\Local\Temp\ipykernel_18480\1648893521.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [314]:
df.head()

,id,time_ts,building,winter_flag,spring_flag,summer_flag,fall_flag,outdoor_temp_c,humidity_pct,cloud_cover_pct,...,day_of_week,month_num,day_of_year,is_weekend,is_holiday,is_peak_hour,lighting_kw,hvac_kw,special_equipment_kw,use_kw
0,2024010100_Hospital,2024-01-01 00:00:00,Hospital,1,0,0,0,11.6,78.0,100.0,...,0,1,1,0,1,0,1.7254,2.3010,6.1852,10.0973
1,2024010101_Hospital,2024-01-01 01:00:00,Hospital,1,0,0,0,12.0,76.0,100.0,...,0,1,1,0,1,0,1.7412,2.4692,5.8280,10.2445
2,2024010102_Hospital,2024-01-01 02:00:00,Hospital,1,0,0,0,12.5,78.0,100.0,...,0,1,1,0,1,0,1.7711,2.9660,5.9904,10.7215
3,2024010103_Hospital,2024-01-01 03:00:00,Hospital,1,0,0,0,12.0,79.0,100.0,...,0,1,1,0,1,0,1.8242,2.7697,6.3425,10.8849
4,2024010104_Hospital,2024-01-01 04:00:00,Hospital,1,0,0,0,11.6,80.0,100.0,...,0,1,1,0,1,0,1.7155,2.7714,6.4427,10.6480


In [315]:
df.groupby("building")["use_kw"].agg(["min", "max", "mean"])

,min,max,mean
building,,,
Hospital,9.4902,18.3249,12.989599
House1,0.5485,3.0566,1.062503
House10,0.4824,1.7851,0.875714
House2,0.5474,2.4582,1.368216
House3,0.7711,3.3594,1.371537
House4,0.5892,2.7967,1.057173
House5,0.5693,1.7164,0.888922
House6,0.6207,1.5484,1.053763
House7,0.4626,2.9740,1.037622


In [308]:
# Drop specified columns
df = df.drop(columns=['lighting_kw', 'hvac_kw', 'special_equipment_kw'])

# Verify the columns have been dropped
print(df.columns)


Index(['id', 'time_ts', 'building', 'winter_flag', 'spring_flag',
       'summer_flag', 'fall_flag', 'outdoor_temp_c', 'humidity_pct',
       'cloud_cover_pct', 'solar_radiation_w_m2', 'hour_of_day', 'day_of_week',
       'month_num', 'day_of_year', 'is_weekend', 'is_holiday', 'is_peak_hour',
       'use_kw'],
      dtype='object')


In [309]:
# Convert 'time_ts' to datetime if not already done
df['time_ts'] = pd.to_datetime(df['time_ts'])


In [218]:
# Split features and target
X = df.drop(columns=['use_kw', 'id', 'time_ts'])  # Drop non-feature columns
y = df['use_kw']

In [219]:
df.shape

(278188, 19)

In [220]:
# Define preprocessing for numerical and categorical features
numerical_features = ['outdoor_temp_c', 'humidity_pct', 'cloud_cover_pct', 
                      'solar_radiation_w_m2', 'hour_of_day', 'day_of_week', 
                      'month_num', 'day_of_year', 'is_weekend', 'is_holiday', 
                      'is_peak_hour']
categorical_features = ['building']


In [221]:
# Numerical preprocessing: Impute missing values, then scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing: Impute missing values, then encode categorical variables
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [222]:
df.shape

(278188, 19)

In [234]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [235]:
# Combine numerical and categorical preprocessing into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Fit and transform the data using the preprocessor
X_transformed = preprocessor.fit_transform(X_train)  # X_train is your training data


In [238]:
from sklearn.ensemble import RandomForestRegressor

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on the preprocessed data
model.fit(X_transformed, y_train)  # y_train is your target variable


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [239]:
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [240]:
joblib.dump(model, 'energy_model.pkl')

['energy_model.pkl']

In [242]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Define the margin (confidence range) as ± MAE
confidence_margin = mae

# Create an array to store whether each prediction is within the margin
within_margin = np.abs(y_pred - y_test) <= confidence_margin

# Calculate the percentage of predictions within the confidence range
accuracy_within_margin = np.mean(within_margin) * 100

print(f'Percentage of predictions within the confidence range: {accuracy_within_margin:.2f}%')



Mean Absolute Error (MAE): 0.15197906249326
Percentage of predictions within the confidence range: 72.81%


In [243]:
# Load the pre-trained model
model = joblib.load('energy_model.pkl')

# Load the preprocessor (scaler and encoder)
preprocessor = joblib.load('preprocessor.pkl')


In [316]:
# Load the 2026-2027 generated data (CSV file)
df = pd.read_csv('C:/Users/Abouda/Desktop/renewstation-huawei-cloud/src/pipeline/generator/data/energy_2026_2027_synthetic_open_meteo.csv')

In [317]:
df.head()

,id,time_ts,building,winter_flag,spring_flag,summer_flag,fall_flag,outdoor_temp_c,humidity_pct,cloud_cover_pct,solar_radiation_w_m2,hour_of_day,day_of_week,month_num,day_of_year,is_weekend,is_holiday,is_peak_hour,use_kw
0,1,2026-01-01 00:00:00,Hospital,1,0,0,0,14.8,81.0,81.0,0.0,0,3,1,1,0,1,0,11.409993
1,2,2026-01-01 01:00:00,Hospital,1,0,0,0,14.7,84.0,18.0,0.0,1,3,1,1,0,1,0,11.908243
2,3,2026-01-01 02:00:00,Hospital,1,0,0,0,13.8,88.0,7.0,0.0,2,3,1,1,0,1,0,11.800598
3,4,2026-01-01 03:00:00,Hospital,1,0,0,0,12.6,90.0,2.0,0.0,3,3,1,1,0,1,0,12.684339
4,5,2026-01-01 04:00:00,Hospital,1,0,0,0,11.5,92.0,1.0,0.0,4,3,1,1,0,1,0,12.505938


In [318]:
df['time_ts'] = pd.to_datetime(df['time_ts'])



In [319]:
X_new = df[['outdoor_temp_c', 'humidity_pct', 'cloud_cover_pct', 'solar_radiation_w_m2', 
            'hour_of_day', 'day_of_week', 'month_num', 'day_of_year', 'is_weekend', 'is_holiday', 'is_peak_hour', 'building']]

In [320]:
X_new

,outdoor_temp_c,humidity_pct,cloud_cover_pct,solar_radiation_w_m2,hour_of_day,day_of_week,month_num,day_of_year,is_weekend,is_holiday,is_peak_hour,building
0,14.8,81.0,81.0,0.0,0,3,1,1,0,1,0,Hospital
1,14.7,84.0,18.0,0.0,1,3,1,1,0,1,0,Hospital
2,13.8,88.0,7.0,0.0,2,3,1,1,0,1,0,Hospital
3,12.6,90.0,2.0,0.0,3,3,1,1,0,1,0,Hospital
4,11.5,92.0,1.0,0.0,4,3,1,1,0,1,0,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...
297835,13.4,83.0,89.0,0.0,19,4,12,365,0,1,1,School
297836,13.3,83.0,96.0,0.0,20,4,12,365,0,1,1,School
297837,13.5,83.0,99.0,0.0,21,4,12,365,0,1,0,School
297838,13.4,82.0,89.0,0.0,22,4,12,365,0,1,0,School


In [321]:
# Apply the preprocessor (scaling and encoding) to this subset of features
X_new_processed = preprocessor.fit_transform(X_new)

In [322]:
X_new_processed.shape


(297840, 28)

In [324]:
pred = model.predict(X_new_processed)

In [325]:
test = df['use_kw']

In [326]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(test, pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Define the margin (confidence range) as ± MAE
confidence_margin = mae

# Create an array to store whether each prediction is within the margin
within_margin = np.abs(pred - test) <= confidence_margin

# Calculate the percentage of predictions within the confidence range
accuracy_within_margin = np.mean(within_margin) * 100

print(f'Percentage of predictions within the confidence range: {accuracy_within_margin:.2f}%')



Mean Absolute Error (MAE): 0.6337472421013484
Percentage of predictions within the confidence range: 70.57%


In [327]:
pred

array([11.438721, 11.429251, 11.10277 , ...,  3.055214,  2.89883 ,
        2.891073], shape=(297840,))

In [328]:
print(test.min(), test.max(), test.mean())


0.6099757313977925 27.354163396443106 4.7786976979809594
